<a href="https://colab.research.google.com/github/JossySoo/natural_language_processing_excercises/blob/main/Notebooks/Desafio_2%20-%20Custom%20embedding%20con%20Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2024-05-24 22:08:54--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2024-05-24 22:08:54--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2024-05-24 22:08:55--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (ra

In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['nicki-minaj.txt',
 'radiohead.txt',
 'bruno-mars.txt',
 'Lil_Wayne.txt',
 'jimi-hendrix.txt',
 'lady-gaga.txt',
 'disney.txt',
 'al-green.txt',
 'dr-seuss.txt',
 'alicia-keys.txt',
 'beatles.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'adele.txt',
 'patti-smith.txt',
 'bjork.txt',
 'blink-182.txt',
 'kanye.txt',
 'johnny-cash.txt',
 'dickinson.txt',
 'cake.txt',
 'michael-jackson.txt',
 'lil-wayne.txt',
 'notorious_big.txt',
 'Kanye_West.txt',
 'britney-spears.txt',
 'lin-manuel-miranda.txt',
 'kanye-west.txt',
 'notorious-big.txt',
 'r-kelly.txt',
 'missy-elliott.txt',
 'paul-simon.txt',
 'bruce-springsteen.txt',
 'joni-mitchell.txt',
 'prince.txt',
 'eminem.txt',
 'dj-khaled.txt',
 'bob-dylan.txt',
 'nirvana.txt',
 'lorde.txt',
 'nursery_rhymes.txt',
 'bieber.txt',
 'nickelback.txt',
 'amy-winehouse.txt',
 'rihanna.txt',
 'janisjoplin.txt',
 'ludacris.txt',
 'drake.txt',
 'dolly-parton.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

<ipython-input-4-91c4bad52ba1>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.15625
Loss after epoch 1: 65953.953125
Loss after epoch 2: 65934.265625
Loss after epoch 3: 65711.96875
Loss after epoch 4: 63872.5
Loss after epoch 5: 64142.46875
Loss after epoch 6: 64058.8125
Loss after epoch 7: 64755.4375
Loss after epoch 8: 62585.5625
Loss after epoch 9: 60415.25
Loss after epoch 10: 59823.625
Loss after epoch 11: 58930.1875
Loss after epoch 12: 57726.4375
Loss after epoch 13: 56493.125
Loss after epoch 14: 55842.5625
Loss after epoch 15: 55862.5
Loss after epoch 16: 51695.8125
Loss after epoch 17: 49815.5
Loss after epoch 18: 49568.125
Loss after epoch 19: 48959.75


(156986, 287740)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087147116661072),
 ('sleep', 0.8649678230285645),
 ('help', 0.8594305515289307),
 ('try', 0.8456019759178162),
 ('cry', 0.8387227654457092),
 ('little', 0.8273633718490601),
 ('not', 0.8181102871894836),
 ('seems', 0.8175203204154968),
 ('twist', 0.8154979348182678),
 ('peace', 0.812263011932373)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.21786975860595703),
 ('our', -0.24495810270309448),
 ('come', -0.2613270878791809),
 ('on', -0.26722386479377747),
 ('bang', -0.27329546213150024),
 ('five', -0.2741836607456207),
 ('six', -0.2774038314819336),
 ('baby', -0.2776353061199188),
 ('work', -0.2790372967720032),
 ('four', -0.2792743146419525)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9786694049835205),
 ('three', 0.9761051535606384),
 ('six', 0.966891348361969),
 ('two', 0.9565720558166504),
 ('seven', 0.954232931137085),
 ('sixty', 0.9003373980522156),
 ('one', 0.816501259803772),
 ('us', 0.7762899398803711),
 ('crying', 0.772737979888916),
 ('strawberry', 0.7672452330589294)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409592747688293),
 ('buy', 0.9399508237838745),
 ('much', 0.8996850848197937),
 ('hide', 0.8465201258659363),
 ('just', 0.8415144085884094)]

In [18]:
# Ensayar con una palabra que no está en el vocabulario:
#w2v_model.wv.most_similar(negative=["diedaa"])

In [19]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736822  0.21460655 -0.05408018  0.06707036  0.13059464 -0.25713933
  0.00711611  0.26743424 -0.03100746  0.259963    0.21179457 -0.08465387
 -0.26272097  0.308386   -0.07934437 -0.13357355  0.2994755   0.18062147
 -0.13111673 -0.11857472  0.03468326  0.1125607   0.08478875  0.08090881
 -0.17622267  0.03122034 -0.08725598  0.23746666 -0.11507358 -0.29119617
 -0.03650665 -0.3322312   0.23262788  0.10366543 -0.18365337  0.08299578
  0.14315338 -0.19761457 -0.00757894  0.1119568  -0.10421463 -0.13083619
 -0.03264419 -0.04687435  0.11330808  0.10231452 -0.16202034 -0.10347028
  0.03659901 -0.08524423 -0.34872082 -0.02154255  0.25938764  0.16540806
  0.0310861   0.09505633  0.24452917 -0.09859891  0.18298703  0.0645447
  0.016967   -0.3259168  -0.04774074 -0.05418535 -0.04338397 -0.09137302
  0.00889874  0.11636022 -0.25662988  0.05426472  0.08915784  0.04541444
  0.20835714 -0.19101828  0.32210803  0.19337991  0.09080223 -0.00362429
 -0.18664253 -0.07053486 -0.1213743  -0.03477469 -0.

In [20]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689683079719543),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385539054870605)]

In [21]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689682483673096),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385538458824158),
 ('hope', 0.8361446261405945)]

### 5 - Visualizar agrupación de vectores

In [22]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [23]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [24]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [25]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

# Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

## 1 -  Preprocesamiento

In [26]:
#import os
#import platform

# Verifica si el archivo PDF ya está descargado
if os.access('./Jane Austen - Orgullo y Prejuicio.pdf', os.F_OK) is False:
    # Verifica si el archivo PDF ya está descargado
    if os.access('orgullo_y_prejuicio.pdf', os.F_OK) is False:
        # Descargar el archivo según el sistema operativo
        if platform.system() == 'Windows':
            !curl -L -o orgullo_y_prejuicio.pdf https://github.com/JossySoo/natural_language_processing_excercises/raw/main/Data/Jane%20Austen%20-%20Orgullo%20y%20Prejuicio.pdf
        else:
            !wget -O orgullo_y_prejuicio.pdf https://github.com/JossySoo/natural_language_processing_excercises/raw/main/Data/Jane%20Austen%20-%20Orgullo%20y%20Prejuicio.pdf
else:
    print("El PDF ya se encuentra descargado")


--2024-05-24 22:09:47--  https://github.com/JossySoo/natural_language_processing_excercises/raw/main/Data/Jane%20Austen%20-%20Orgullo%20y%20Prejuicio.pdf
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JossySoo/natural_language_processing_excercises/main/Data/Jane%20Austen%20-%20Orgullo%20y%20Prejuicio.pdf [following]
--2024-05-24 22:09:47--  https://raw.githubusercontent.com/JossySoo/natural_language_processing_excercises/main/Data/Jane%20Austen%20-%20Orgullo%20y%20Prejuicio.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 609042 (595K) [application/octet-stream]
Saving to: ‘orgullo_y_prej

In [27]:
print("Tamaño del archivo descargado:", os.path.getsize("orgullo_y_prejuicio.pdf"), "bytes")

Tamaño del archivo descargado: 609042 bytes


In [28]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 64.5 MB/s eta 0:00:00


In [29]:
import fitz  # PyMuPDF

# Leer el PDF y extraer el texto
pdf_document = "orgullo_y_prejuicio.pdf"
doc = fitz.open(pdf_document)
text = ""
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text += page.get_text()

# Mostrar el texto extraído (opcional)
print(text[:1000])  # Muestra los primeros 1000 caracteres

# Guardar el texto extraído en un archivo de texto
with open("orgullo_y_prejuicio.txt", "w", encoding="utf-8") as text_file:
    text_file.write(text)


Orgullo y Prejuicio
Jane Austen
textos.info
Biblioteca digital abierta
1
Texto núm. 3006
Título: Orgullo y Prejuicio
Autor: Jane Austen
Etiquetas: Novela
Editor: Edu Robsy
Fecha de creación: 7 de noviembre de 2017
Fecha de modificación: 7 de noviembre de 2017
Edita textos.info
Maison Carrée
c/ Ramal, 48
07730 Alayor - Menorca
Islas Baleares
España
Más textos disponibles en http://www.textos.info
2
Capítulo I
Es una verdad mundialmente reconocida que un hombre soltero, poseedor 
de una gran fortuna, necesita una esposa.
Sin embargo, poco se sabe de los sentimientos u opiniones de un hombre 
de tales condiciones cuando entra a formar parte de un vecindario. Esta 
verdad está tan arraigada en las mentes de algunas de las familias que lo 
rodean, que algunas le consideran de su legítima propiedad y otras de la 
de sus hijas.
—Mi querido señor Bennet —le dijo un día su esposa—, ¿sabías que, por 
fin, se ha alquilado Netherfield Park?
El señor Bennet respondió que no.
—Pues así es —insistió 

In [30]:
import re

# Leer el archivo de texto
with open("orgullo_y_prejuicio.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Limpieza básica del texto
text = text.lower()
text = re.sub(r'\s+', ' ', text)  # Reemplaza múltiples espacios por uno solo
text = re.sub(r'[^a-záéíóúüñ\s.?!]', '', text)  # Elimina caracteres no alfabéticos excepto vocales con tildes y eñes

# Tokenizar el texto en oraciones usando el punto como delimitador
sentences = re.split(r'(?<=[.?!])\s+', text)

# Guardar cada oración en una nueva línea
with open("orgullo_y_prejuicio_clean.txt", "w", encoding="utf-8") as text_file:
    for sentence in sentences:
        text_file.write(sentence + '\n')


In [31]:
# Leer el archivo de texto línea por línea usando pandas
df = pd.read_csv('orgullo_y_prejuicio_clean.txt', sep='/n', header=None, names=['sentence'])
print(df.head(25))

                                             sentence
0   orgullo y prejuicio jane austen textos.info bi...
1   título orgullo y prejuicio autor jane austen e...
2   sin embargo poco se sabe de los sentimientos u...
3   esta verdad está tan arraigada en las mentes d...
4   mi querido señor bennet le dijo un día su espo...
5                   el señor bennet respondió que no.
6   pues así es insistió ella la señora long ha es...
7        el señor bennet no hizo ademán de contestar.
8             no quieres saber quién lo ha alquilado?
9                            se impacientó su esposa.
10  eres tú la que quieres contármelo y yo no teng...
11                 esta sugerencia le fue suficiente.
12  pues sabrás querido que la señora long dice qu...
13                                     cómo se llama?
14                                           bingley.
15                             está casado o soltero?
16                                                oh!
17                      solt

<ipython-input-31-1cab1fe38691>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [32]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 6185


In [33]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [34]:
sentence_tokens[:5]

[['orgullo',
  'y',
  'prejuicio',
  'jane',
  'austen',
  'textos',
  'info',
  'biblioteca',
  'digital',
  'abierta',
  'texto',
  'núm'],
 ['título',
  'orgullo',
  'y',
  'prejuicio',
  'autor',
  'jane',
  'austen',
  'etiquetas',
  'novela',
  'editor',
  'edu',
  'robsy',
  'fecha',
  'de',
  'creación',
  'de',
  'noviembre',
  'de',
  'fecha',
  'de',
  'modificación',
  'de',
  'noviembre',
  'de',
  'edita',
  'textos',
  'info',
  'maison',
  'carrée',
  'c',
  'ramal',
  'alayor',
  'menorca',
  'islas',
  'baleares',
  'españa',
  'más',
  'textos',
  'disponibles',
  'en',
  'httpwww',
  'textos',
  'info',
  'capítulo',
  'i',
  'es',
  'una',
  'verdad',
  'mundialmente',
  'reconocida',
  'que',
  'un',
  'hombre',
  'soltero',
  'poseedor',
  'de',
  'una',
  'gran',
  'fortuna',
  'necesita',
  'una',
  'esposa'],
 ['sin',
  'embargo',
  'poco',
  'se',
  'sabe',
  'de',
  'los',
  'sentimientos',
  'u',
  'opiniones',
  'de',
  'un',
  'hombre',
  'de',
  'tales',

## 2 - Crear los vectores (word2vec)

In [35]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [36]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=100,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [37]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [38]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 6185


In [39]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 2141


## 3 - Entrenar embeddings

In [41]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
np.random.seed(55)
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 903263.1875
Loss after epoch 1: 647614.6875
Loss after epoch 2: 608220.625
Loss after epoch 3: 528824.0
Loss after epoch 4: 526748.0
Loss after epoch 5: 523616.0
Loss after epoch 6: 515840.5
Loss after epoch 7: 486603.0
Loss after epoch 8: 483531.5
Loss after epoch 9: 479954.0
Loss after epoch 10: 474597.5
Loss after epoch 11: 474081.0
Loss after epoch 12: 469917.0
Loss after epoch 13: 463764.0
Loss after epoch 14: 461409.5
Loss after epoch 15: 454052.5
Loss after epoch 16: 435491.0
Loss after epoch 17: 433866.0
Loss after epoch 18: 431078.0
Loss after epoch 19: 429495.0
Loss after epoch 20: 424878.0
Loss after epoch 21: 424384.0
Loss after epoch 22: 422503.0
Loss after epoch 23: 418818.0
Loss after epoch 24: 417541.0
Loss after epoch 25: 416008.0
Loss after epoch 26: 414963.0
Loss after epoch 27: 412059.0
Loss after epoch 28: 410941.0
Loss after epoch 29: 410182.0
Loss after epoch 30: 410249.0
Loss after epoch 31: 406825.0
Loss after epoch 32: 405570.0
Loss after e

(3343706, 5756900)

## 4 - Ensayar

In [42]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["prejuicio"], topn=10)

[('siente', 0.6261991262435913),
 ('indignación', 0.5521978735923767),
 ('significaba', 0.5480401515960693),
 ('influencia', 0.5460621118545532),
 ('abominable', 0.5404924750328064),
 ('pariente', 0.5353097319602966),
 ('profunda', 0.5310643911361694),
 ('dignidad', 0.5240452289581299),
 ('consejo', 0.5206248760223389),
 ('modestia', 0.5198466181755066)]

In [43]:
w2v_model.wv.most_similar(positive=["darcy"], topn=10)

[('y', 0.5229238867759705),
 ('administrador', 0.5095698833465576),
 ('jones', 0.5032007098197937),
 ('bennet', 0.49783971905708313),
 ('bingley', 0.49715736508369446),
 ('antipático', 0.49289530515670776),
 ('su', 0.4908480644226074),
 ('sonriendo', 0.47856229543685913),
 ('admiración', 0.4769458770751953),
 ('éste', 0.47225087881088257)]

In [44]:
w2v_model.wv.most_similar(positive=["amor"], topn=10)

[('afecto', 0.5186800360679626),
 ('éxito', 0.5041104555130005),
 ('cariño', 0.49677205085754395),
 ('acordarse', 0.4786473214626312),
 ('mío', 0.4772559404373169),
 ('verdadero', 0.4742584526538849),
 ('fuerte', 0.4731135070323944),
 ('significado', 0.47126057744026184),
 ('administrador', 0.4578981101512909),
 ('sincero', 0.45083850622177124)]

In [45]:
w2v_model.wv.most_similar(positive=["matrimonio"], topn=10)

[('posee', 0.4448643922805786),
 ('bienestar', 0.4430423676967621),
 ('unión', 0.44038403034210205),
 ('insistió', 0.4402417838573456),
 ('futuro', 0.4388976991176605),
 ('habitual', 0.4313914477825165),
 ('defecto', 0.42915287613868713),
 ('promesa', 0.4274199306964874),
 ('obtenido', 0.4258091449737549),
 ('hallarse', 0.4189453721046448)]

In [46]:
w2v_model.wv.most_similar(positive=["hermanas"], topn=10)

[('menores', 0.5551480054855347),
 ('partes', 0.4889084994792938),
 ('respetos', 0.4803032875061035),
 ('molesta', 0.47938498854637146),
 ('veras', 0.46616965532302856),
 ('advertencias', 0.4630378186702728),
 ('amigas', 0.4630282521247864),
 ('muchachas', 0.4628240764141083),
 ('james', 0.45952293276786804),
 ('expresiones', 0.45742520689964294)]

In [47]:
w2v_model.wv.most_similar(positive=["mujer"], topn=10)

[('compañero', 0.4947425127029419),
 ('ejercicio', 0.4451231062412262),
 ('casar', 0.4433146119117737),
 ('pretende', 0.4376656413078308),
 ('independencia', 0.434664785861969),
 ('criatura', 0.43154439330101013),
 ('atenta', 0.42921003699302673),
 ('superior', 0.42809510231018066),
 ('fácil', 0.42655882239341736),
 ('imaginación', 0.42630070447921753)]

Vemos que varias de las palabras que se muestran como similares tienen sentido en el contexto de la novel Orgullo y prejuicio. Por ejemplo "Darcy" lo relaciona con otros apellidos cono "Bingley" o "Bennet". "Amor" también lo relaciona con "afecto","cariño". Además refleja las ideas que se discuten en el libro,como el rol de la "mujer", ya que relaciona la palabra con "casar", "compañero" (por el hecho que se discute en el libro de que las mujeres en esa época deben buscar esposo), "independencia".
No todas las palabras a calzan exáctamente, pero está bastante bien el resultado.

## 5 - Graficar

In [48]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [49]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Vemos en el gráfico que palabras muy cercanas son aquellas que se usan juntas, como "lady" "Catherine". También relacionó correctamente "hermanas" con "hijas"

In [50]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

En el gráfico encontramos otras relaciones que hacen sentido, como "Lydia"con "hija", o "padre", "amigo" e "hijo" a la misma altura. "Mujer" y "hemana" también están paralelas en un eje.